# [WUM] Praca Domowa nr7
## Kacper Kurowski

Załadujmy wpierw zbiór danych

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture
from sklearn import preprocessing
import seaborn as sns
import sklearn

In [ ]:
val = pd.read_csv( "val.csv")
train = pd.read_csv( "train.csv")
test = pd.read_csv( "test.csv")
train_vals = [0]*len( train)

i pobieżnie go obejrzmy

In [ ]:
val.head()

In [ ]:
train.head()

In [ ]:
test.head()

Postarajmy się znaleźć optymalne parametry dla GMM

In [ ]:
import numpy as np
import itertools

from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import mixture

In [ ]:
X = train

bics = [ [] for i in range( 30)]

lowest_bic = np.infty
bic = []
n_components_range = range(1, 10)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        for t in range(30): # Powtórzmy eksperyment wielokrotnie, dla bardziej miarodajnych wyników
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,
                                      covariance_type=cv_type)
            gmm.fit(X, y = train_vals)
            bics[t] = gmm.bic(X)
        bic.append( np.mean( np.array( bics)))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                              'darkorange'])
clf = best_gmm
bars = []

# Plot the BIC scores
plt.figure(figsize=(8, 6))
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

plt.show()

W przypadku BIC score im mniejsza wartość, tym lepiej. Okazuje się, że najlepszy wynik dały 2 komponenty z diagonalną kowariancją

In [ ]:
gmm = mixture.GaussianMixture(
    n_components=2,
    covariance_type='diag'
).fit( train, y = train_vals)

In [ ]:
test_pred = gmm.predict( test.drop(['class'], axis=1))
test_true = test['class'].to_numpy()

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
print( "f1_score: % 4.3f" % f1_score( test_true, test_hat))
print( "precision_score: % 4.3f" % precision_score( test_true, test_hat))
print( "recall_score: % 4.3f" % recall_score( test_true, test_hat))

Zatem, wyniki nie są bardzo wysokie, jednakże, na podstawie recall, wiemy, że gmm był w stanie znaleźć wszystkie outliery. Niestety, wiele inlierowych wartości zakwalifikował jako outliery, co wiemy chociażby na podstawie precision.